In [2]:
import pandas as pd

#Load csv files
school_csv = "../Resources/schools_complete.csv"
student_csv = "../Resources/students_complete.csv"

#Read School and Student csv files
school_read = pd.read_csv(school_csv)
student_read = pd.read_csv(student_csv)

#Combine data files into single dataset and dataframe
school_data_df = pd.merge(student_read, school_read, how= 'left', on=['school_name', 'school_name'])
school_data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


District Summary

In [36]:
school_count = school_data_df['school_name'].nunique()
school_count


15

In [41]:
student_count = school_data_df['student_name'].value_counts()
student_total = student_count.sum()
student_total

39170

In [54]:
total_budget = school_data_df['budget'].drop_duplicates()
total = total_budget.sum()
total


24649428

In [55]:
average_math_score = school_data_df['math_score'].mean()
average_math_score

78.98537145774827

In [57]:
average_reading_score = school_data_df['reading_score'].mean()
average_reading_score

81.87784018381414

In [58]:
passing_math_count = school_data_df[(school_data_df['math_score'] >= 70)].count()['student_name']
mathPass_percent = passing_math_count / float(student_total) * 100
mathPass_percent

74.9808526933878

In [70]:
passing_reading_count = school_data_df[(school_data_df['reading_score'] >= 70)].count()['student_name']
readPass_percent = passing_reading_count / float(student_total) * 100
readPass_percent

85.80546336482001

In [71]:
overall_pass_count = school_data_df[(school_data_df['math_score'] >= 70) & 
                                    (school_data_df['reading_score']>= 70)].count()['student_name']
overall_passing = overall_pass_count / float(student_total) * 100
overall_passing

65.17232575950983

In [72]:
district_summary = [{'Total Schools': school_count, 'Total Students': student_total, 'Total Budget': total,
                     'Average Math Score': average_math_score, 'Average Reading Score': average_reading_score,
                     '% Passing Math': mathPass_percent, '% Passing Reading': readPass_percent, '% Overall Passing': overall_passing}]

district_summary = pd.DataFrame(district_summary)

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


School Summary